In [1]:
pip install optimum[onnxruntime]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers[torch]

In [3]:
!pip install datasets


In [4]:
pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [5]:
!pip install accelerate

In [6]:
from datasets import load_dataset
import pandas as pd
import re

# Dataset links
datasets = {
    "sentiment_train": "lelapa/SentimentTrain",
    #"sentiment_test": "lelapa/SentimentTest",
    "xnli_train": "lelapa/XNLITrain",
    #"xnli_test": "lelapa/XNLITest",
    "mt_train": "lelapa/MTTrain",
    #"mt_test": "lelapa/MTTest",
}

# Function to clean strings
def clean_string(text):
    """
    Cleans the string by removing unwanted characters and fixing extra spaces or empty strings.
    """
    if not isinstance(text, str):
        return text  # Skip if it's not a string
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Function to clean and load dataset
def load_and_clean_dataset(dataset_name, task_type):
    """
    Downloads and cleans the dataset, returns a cleaned DataFrame.
    """
    # Load dataset from Hugging Face
    dataset = load_dataset(dataset_name)
    # Convert dataset to pandas DataFrame
    if "train" in dataset:
        df = pd.DataFrame(dataset["train"])
    elif "test" in dataset:
        df = pd.DataFrame(dataset["test"])
    else:
        raise ValueError("Dataset type (train/test) not found.")

    # Apply cleaning to all string columns
    for column in df.columns:
        if df[column].dtype == "object":  # Clean only string columns
            df[column] = df[column].apply(clean_string)

    # Drop empty rows caused by cleaning
    df = df.dropna().reset_index(drop=True)

    # Add a task column for identification
    df["task"] = task_type
    return df

# Download and clean datasets
sentiment_train_df = load_and_clean_dataset(datasets["sentiment_train"], task_type="sentiment")
#sentiment_test_df = load_and_clean_dataset(datasets["sentiment_test"], task_type="sentiment")
xnli_train_df = load_and_clean_dataset(datasets["xnli_train"], task_type="xnli")
#xnli_test_df = load_and_clean_dataset(datasets["xnli_test"], task_type="xnli")
mt_train_df = load_and_clean_dataset(datasets["mt_train"], task_type="translation")
#mt_test_df = load_and_clean_dataset(datasets["mt_test"], task_type="translation")

# Combine all datasets
combined_df = pd.concat(
    [sentiment_train_df, xnli_train_df, mt_train_df],
    ignore_index=True,
)

# Save the cleaned and combined dataset
combined_df.to_csv("cleaned_combined_dataset.csv", index=False)
print("Combined dataset saved as 'cleaned_combined_dataset.csv'.")

# Display a sample of the combined dataset
print(combined_df.head())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/447 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/72.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Combined dataset saved as 'cleaned_combined_dataset.csv'.
                                   ID       task    langs    data_source  \
0    ID_6aba33a1_sentiment_ dev_hausa  sentiment    hausa      afrisenti   
1    ID_ce64d307_sentiment_ dev_hausa  sentiment    hausa     naijasenti   
2  ID_dfb02831_sentiment_ dev_swahili  sentiment  swahili  swahili_tweet   
3    ID_2efc9515_sentiment_ dev_hausa  sentiment    hausa      afrisenti   
4  ID_ad1d9888_sentiment_ dev_swahili  sentiment  swahili      afrisenti   

                                         instruction  \
0  Za ka iya tantance yanayin wannan rubutu Bi wa...   
1  Da fatan za a gano raayin da ke cikin wannan r...   
2  Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...   
3  Gano raayin da aka bayyana a cikin wannan rubu...   
4  Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...   

                                              inputs     targets premise  
0                user user allah ya tsayyaba yar uwa   Kyakkyawa     NaN 

In [7]:
from datasets import load_dataset
import pandas as pd

# Dataset links
datasets = {
    "sentiment_test": "lelapa/SentimentTest",
    "xnli_test": "lelapa/XNLITest",
    "mt_test": "lelapa/MTTest",
}

# Download and clean datasets (remove task_type)
sentiment_test_df = load_dataset(datasets["sentiment_test"])["train"]
xnli_test_df = load_dataset(datasets["xnli_test"])["train"]
mt_test_df = load_dataset(datasets["mt_test"])["train"]

# Convert Hugging Face Dataset objects to Pandas DataFrames
sentiment_test_df = pd.DataFrame(sentiment_test_df)
xnli_test_df = pd.DataFrame(xnli_test_df)
mt_test_df = pd.DataFrame(mt_test_df)

# Combine all datasets
combined_test_df1 = pd.concat([sentiment_test_df, xnli_test_df, mt_test_df], ignore_index=True)

# Save the cleaned and combined dataset
combined_test_df1.to_csv("cleaned_test_dataset1.csv", index=False)
print("✅ Combined dataset saved as 'cleaned_test_dataset1.csv'.")

# Display a sample of the combined dataset
print(combined_test_df1.head())


README.md:   0%|          | 0.00/486 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/447 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/484 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/22.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

✅ Combined dataset saved as 'cleaned_test_dataset1.csv'.
                                    ID       task    langs data_source  \
0    ID_f3c74c7b_sentiment_test__hausa  sentiment    hausa   afrisenti   
1    ID_aad19dbf_sentiment_test__hausa  sentiment    hausa  naijasenti   
2    ID_f6de0381_sentiment_test__hausa  sentiment    hausa   afrisenti   
3  ID_cbec84fe_sentiment_test__swahili  sentiment  swahili   afrisenti   
4    ID_885caf5c_sentiment_test__hausa  sentiment    hausa  naijasenti   

                                         instruction  \
0  Gano ra'ayin da aka bayyana a cikin wannan rub...   
1  Za ka iya tantance yanayin wannan rubutu? Bi w...   
2  Za ka iya tantance yanayin wannan rubutu? Bi w...   
3  Changanua mawazo ya matini yanayofuata na uain...   
4  Tantance ra’ayin wannan rubutu kuma a rarraba ...   

                                              inputs targets premise  
0  @user ynxu fha da kanada kudi shikenan duk kay...    None     NaN  
1  @user alhamdulil

In [8]:
# Load a sample of the dataset
dataset = load_dataset('lelapa/SentimentTest')  # Replace with the appropriate dataset link

# Inspect the dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets'],
        num_rows: 300
    })
})


In [9]:
# Load a sample of the dataset
dataset = load_dataset("lelapa/XNLITrain")  # Replace with the appropriate dataset link

# Inspect the dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets'],
        num_rows: 400
    })
})


In [10]:
# Get column names of the dataset
print(dataset['train'].column_names)


['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets']


In [11]:
# Inspect Sentiment dataset labels
print(set(dataset["train"]["targets"]))  # Should output distinct labels like [0, 1]

# Inspect XNLI dataset labels
print(set(dataset["train"]["targets"]))  # Should output distinct labels like [0, 1]


{0, 1, 2}
{0, 1, 2}


In [12]:
from datasets import load_dataset

# Load datasets
sentiment_dataset = load_dataset("lelapa/SentimentTrain")
xnli_dataset = load_dataset("lelapa/XNLITrain")

# Preprocess Sentiment dataset
def preprocess_sentiment(example):
    return {
        "inputs": example["inputs"],  # Input text
        "labels": example["targets"],  # Sentiment label
    }

sentiment_preprocessed = sentiment_dataset.map(preprocess_sentiment)

# Preprocess XNLI dataset
def preprocess_xnli(example):
    return {
        "inputs": example["inputs"],  # Input hypothesis
        "labels": example["targets"],  # XNLI label (e.g., true/false)
    }

xnli_preprocessed = xnli_dataset.map(preprocess_xnli)

# Inspect the processed datasets
print(sentiment_preprocessed)
print(xnli_preprocessed)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'task', 'langs', 'data_source', 'instruction', 'inputs', 'targets', 'labels'],
        num_rows: 400
    })
})
DatasetDict({
    train: Dataset({
        features: ['ID', 'langs', 'premise', 'inputs', 'instruction', 'targets', 'labels'],
        num_rows: 400
    })
})


In [13]:
from datasets import load_dataset, concatenate_datasets

# Load datasets
sentiment_dataset = load_dataset("lelapa/SentimentTrain")
xnli_dataset = load_dataset("lelapa/XNLITrain")

sentiment_label_map = {
    "Hasi": 0,  # Example: Negative
    "Korau": 0,  # Example: Negative
    "Tsaka-tsaki": 1,  # Example: Neutral
    "Wastani": 1,  # Example: Neutral
    "Chanya": 2,  # Example: Positive
    "Kyakkyawa": 2   # Example: Positive
}


# Ensure all features have consistent types
def preprocess_sentiment(example):
    return {
        "ID": str(example["ID"]),
        "inputs": example["inputs"],  # Input text
        "labels": sentiment_label_map.get(example["targets"], -1),  # Convert text to int
        "task": "sentiment"
    }

def preprocess_xnli(example):
    return {
        "ID": str(example["ID"]),
        "inputs": example["inputs"],  # Input hypothesis
        "labels": int(example["targets"]),  # Convert to int
        "task": "xnli"
    }

# Apply preprocessing
sentiment_preprocessed = sentiment_dataset["train"].map(preprocess_sentiment)
xnli_preprocessed = xnli_dataset["train"].map(preprocess_xnli)

# Convert to same column structure
sentiment_preprocessed = sentiment_preprocessed.remove_columns(["targets", "data_source", "langs", "instruction"])
xnli_preprocessed = xnli_preprocessed.remove_columns(["targets", "langs", "premise", "instruction"])

# Ensure the column order is identical
sentiment_preprocessed = sentiment_preprocessed.rename_columns({"inputs": "text"})
xnli_preprocessed = xnli_preprocessed.rename_columns({"inputs": "text"})

# Combine datasets
combined_dataset = concatenate_datasets([sentiment_preprocessed, xnli_preprocessed])

# Shuffle the dataset
combined_dataset = combined_dataset.shuffle(seed=42)

# Inspect the final combined dataset
print(combined_dataset)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['ID', 'task', 'text', 'labels'],
    num_rows: 800
})


In [14]:
print(set(sentiment_dataset["train"].map(preprocess_sentiment)["labels"]))


{0, 1, 2}


In [15]:
print(set(sentiment_dataset["train"]["targets"]))  # Check unique values
print(set(xnli_dataset["train"]["targets"]))  # Check unique values


{'Tsaka-tsaki', 'Hasi', 'Kyakkyawa', 'Chanya', 'Korau', 'Wastani'}
{0, 1, 2}


In [16]:
from datasets import DatasetDict

# Split dataset (80% train, 20% validation)
split_dataset = combined_dataset.train_test_split(test_size=0.2, seed=42)

# Show dataset sizes
print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['ID', 'task', 'text', 'labels'],
        num_rows: 640
    })
    test: Dataset({
        features: ['ID', 'task', 'text', 'labels'],
        num_rows: 160
    })
})


In [17]:
from transformers import AutoTokenizer

# Load tokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
tokenized_datasets = split_dataset.map(tokenize_function, batched=True)

# Remove the original text column
tokenized_datasets = tokenized_datasets.remove_columns(["text", "ID", "task"])

# Rename "labels" column to "label" for compatibility
tokenized_datasets = tokenized_datasets.rename_column("labels", "label")

# Convert dataset to PyTorch format
tokenized_datasets.set_format("torch")

print(tokenized_datasets)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
})


In [18]:
# Define task-specific tokens
task_tokens = {
    "sentiment": "<sentiment>",
    "xnli": "<xnli>"
}

# Update tokenizer to include new tokens
tokenizer.add_tokens(list(task_tokens.values()))

# Tokenization function with batch processing
def tokenize_function(examples):
    task_tokens_list = [task_tokens[task] for task in examples["task"]]  # Convert all task names to tokens
    text_with_task = [task_token + " " + text for task_token, text in zip(task_tokens_list, examples["text"])]  # Prepend task token
    return tokenizer(text_with_task, truncation=True, padding="max_length", max_length=128)

# Apply tokenization
tokenized_datasets = split_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns(["text", "ID", "task"])

# Rename "labels" column to "label"
tokenized_datasets = tokenized_datasets.rename_column("labels", "label")

# Convert dataset to PyTorch format
tokenized_datasets.set_format("torch")

print(tokenized_datasets)



Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
})


In [19]:
import torch
from transformers import AutoModelForSequenceClassification

# Load transformer model with classification head
num_labels = 3  # Both tasks have 3 classes
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# Resize token embeddings to accommodate new task tokens
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119549, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [20]:
!pip install evaluate


In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [22]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate  # Use evaluate instead of datasets.load_metric

# Load accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./multi_task_model",
    eval_strategy="epoch",  # Updated
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.096228,0.393750
2,No log,1.069560,0.387500
3,No log,1.059693,0.412500
4,No log,1.073665,0.437500
5,No log,1.081945,0.412500


TrainOutput(global_step=200, training_loss=1.0443978881835938, metrics={'train_runtime': 168.2568, 'train_samples_per_second': 19.019, 'train_steps_per_second': 1.189, 'total_flos': 210490734182400.0, 'train_loss': 1.0443978881835938, 'epoch': 5.0})

In [23]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 1.0736653804779053, 'eval_accuracy': 0.4375, 'eval_runtime': 1.1055, 'eval_samples_per_second': 144.734, 'eval_steps_per_second': 9.046, 'epoch': 5.0}


In [24]:
from transformers import Trainer
import torch
import numpy as np

# Make predictions on the test dataset
predictions_output = trainer.predict(tokenized_datasets["test"])

# Extract logits and convert to class predictions
logits = predictions_output.predictions
predicted_classes = np.argmax(logits, axis=-1)

# Extract actual labels for comparison (optional)
actual_labels = tokenized_datasets["test"]["label"]

# Print sample predictions
for i in range(10):  # Show first 10 predictions
    print(f"Predicted: {predicted_classes[i]}, Actual: {actual_labels[i]}")


Predicted: 1, Actual: 1
Predicted: 2, Actual: 1
Predicted: 0, Actual: 0
Predicted: 2, Actual: 2
Predicted: 1, Actual: 1
Predicted: 1, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 2, Actual: 0
Predicted: 1, Actual: 1


In [25]:
import pandas as pd
import torch
import numpy as np
from transformers import Trainer

# Load the test dataset from CSV
test_df = pd.read_csv("/content/cleaned_test_dataset1.csv")

# Ensure 'ID' column exists for submission
if "ID" not in test_df.columns:
    raise ValueError("The test dataset must contain an 'ID' column for submission.")

# Check if 'inputs' column exists
if "inputs" not in test_df.columns:
    raise ValueError("The test dataset must contain an 'inputs' column for text data.")

# Tokenize the test dataset using 'inputs' column
inputs = tokenizer(test_df["inputs"].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Move tensors to the correct device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make predictions
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Extract logits

# Convert logits to class predictions
predicted_classes = np.argmax(logits.cpu().numpy(), axis=-1)

# Create submission DataFrame
submission_df = pd.DataFrame({
    "ID": test_df["ID"],  # Ensure correct ID mapping
    "Response": predicted_classes  # Predictions
})

# Save submission file
submission_df.to_csv("submission1.csv", index=False)
print("✅ Submission file saved as 'submission.csv'.")


✅ Submission file saved as 'submission.csv'.


In [27]:
sub1 = pd.read_csv("/content/submission1.csv")
print(sub1.head(-10), sub1.info(), sub1.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        900 non-null    object
 1   Response  900 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.2+ KB
                                      ID  Response
0      ID_f3c74c7b_sentiment_test__hausa         1
1      ID_aad19dbf_sentiment_test__hausa         2
2      ID_f6de0381_sentiment_test__hausa         1
3    ID_cbec84fe_sentiment_test__swahili         1
4      ID_885caf5c_sentiment_test__hausa         0
..                                   ...       ...
885         ID_40d90780_test__mt_eng-hau         0
886         ID_e41159bb_test__mt_eng-swa         2
887         ID_596cb82e_test__mt_eng-swa         2
888         ID_7e5bf127_test__mt_eng-hau         2
889         ID_68220239_test__mt_eng-swa         2

[890 rows x 2 columns] None (900, 2)


In [28]:
sub = pd.read_csv("/content/cleaned_test_dataset1.csv")
print(sub.head(15), sub.info(), sub.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           900 non-null    object 
 1   task         600 non-null    object 
 2   langs        900 non-null    object 
 3   data_source  600 non-null    object 
 4   instruction  900 non-null    object 
 5   inputs       900 non-null    object 
 6   targets      0 non-null      float64
 7   premise      300 non-null    object 
dtypes: float64(1), object(7)
memory usage: 56.4+ KB
                                     ID       task    langs    data_source  \
0     ID_f3c74c7b_sentiment_test__hausa  sentiment    hausa      afrisenti   
1     ID_aad19dbf_sentiment_test__hausa  sentiment    hausa     naijasenti   
2     ID_f6de0381_sentiment_test__hausa  sentiment    hausa      afrisenti   
3   ID_cbec84fe_sentiment_test__swahili  sentiment  swahili      afrisenti   
4     ID_885caf5c_sentiment_test_

In [29]:
print(test_df[["ID", "inputs"]].head())


                                    ID  \
0    ID_f3c74c7b_sentiment_test__hausa   
1    ID_aad19dbf_sentiment_test__hausa   
2    ID_f6de0381_sentiment_test__hausa   
3  ID_cbec84fe_sentiment_test__swahili   
4    ID_885caf5c_sentiment_test__hausa   

                                              inputs  
0  @user ynxu fha da kanada kudi shikenan duk kay...  
1  @user alhamdulillah babu abinda zamuce sai god...  
2     @user ke ina ruwan ki 😬 ba harkar film bane ba  
3  matokeo chanya liverais magufuli katika uzindu...  
4            @user 🤣 akwai lauje cikin nadi gaskiya.  


In [30]:
import torch
import torch.nn.utils.prune as prune

# Prune weights of the model
def apply_pruning(model, amount=0.2):  # Prune 20% of the weights
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):  # Apply pruning only to Linear layers
            prune.l1_unstructured(module, name="weight", amount=amount)
            prune.remove(module, "weight")  # Make pruning permanent

apply_pruning(model)  # Apply pruning


In [31]:
def get_model_size(model):
    torch.save(model.state_dict(), "temp.pth")
    size_mb = os.path.getsize("temp.pth") / 1e6
    os.remove("temp.pth")
    return size_mb

print(f"Model size before pruning: {get_model_size(model)} MB")
apply_pruning(model, amount=0.2)
print(f"Model size after pruning: {get_model_size(model)} MB")


Model size before pruning: 711.497337 MB
Model size after pruning: 711.497337 MB


In [32]:
import torch.quantization

# Convert model to evaluation mode before quantization
model.eval()

# Apply dynamic quantization (good for Transformer-based models)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8  # Convert only Linear layers to int8
)

# Save the quantized model
torch.save(quantized_model.state_dict(), "quantized_model.pth")

print(f"Model size after quantization: {get_model_size(quantized_model)} MB")


Model size after quantization: 454.974293 MB


In [35]:
import time
import torch
import torch.quantization

def benchmark_model(model, test_input, device):
    model.to(device)  # Move model to the correct device
    model.eval()

    # Ensure input is on the same device as the model
    test_input = {key: value.to(device) for key, value in test_input.items()}

    with torch.no_grad():
        start_time = time.time()
        _ = model(**test_input)
        end_time = time.time()

    return end_time - start_time

# Detect CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the **original** model to GPU (if available)
model.to(device)

# Ensure the quantized model is **only on CPU**
quantized_device = torch.device("cpu")

# 🔹 Re-quantize to ensure compatibility
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},  # Apply quantization to Linear layers
    dtype=torch.qint8    # Use int8 quantization
)

quantized_model.to(quantized_device)

# Create a test input
test_input = tokenizer("This is a test sentence.", return_tensors="pt")

# Measure inference time
print(f"Original Model Inference Time (on {device}): {benchmark_model(model, test_input, device)} seconds")
print(f"Quantized Model Inference Time (on CPU): {benchmark_model(quantized_model, test_input, quantized_device)} seconds")




Original Model Inference Time (on cpu): 0.10543584823608398 seconds
Quantized Model Inference Time (on CPU): 0.06258034706115723 seconds


In [36]:
trainer.model = quantized_model  # Replace model in trainer
results = trainer.evaluate()
print("Post-Quantization Accuracy:", results["eval_accuracy"])


Post-Quantization Accuracy: 0.3875


In [33]:
import torch
import torch.onnx

# Dummy input for tracing (adjust shape based on your model's expected input)
dummy_input = tokenizer("This is a test sentence.", return_tensors="pt")
dummy_input = {k: v.to(device) for k, v in dummy_input.items()} #add attention mask to dummy input

# Ensure model is in eval mode
model.eval()

# Export model to ONNX
torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']), # Pass both input_ids and attention_mask
    "model.onnx",
    export_params=True,
    opset_version=14,   # Change opset_version to 14
    do_constant_folding=True,
    input_names=["input_ids", "attention_mask"], # update input names
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size"},"attention_mask":{0:"batch_size"}, "output": {0: "batch_size"}} #update dynamic_axes
)

print("✅ ONNX model saved as model.onnx")


✅ ONNX model saved as model.onnx


In [34]:
!pip install nvidia-pyindex

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=3cb8b2a2334ce26b0c5d030428fa0c9b22600d30e3ddcf25721edea473f0a5ee
  Stored in directory: /root/.cache/pip/wheels/49/d0/7d/b68b3665d16ee20355e65fb7ef48b7ca26533217d9f09924fe
Successfully built nvidia-pyindex


In [35]:
!pip install onnx onnx_graphsurgeon tensorrt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.6 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.8.0.43-py2.py3-none-any.whl size=42246 sha256=69dd51a6838cdc0c7b212fba6457a98d54ed5b50fa74c525bb6db3b09551f4df
  Stored in directory: /root/.cache/pip/wheels/74/b9/22/cd396c2ef84d0f65b7ad3c218d00b8049e426d0fde196209bc
  Created wheel for tensorrt_cu12: filename=tensorrt_cu12-10.8.0.43-py2.py3-none-any.whl size=17595 sha256=dd26a4d5f56eef2f6679e5dfd5af56e1bb9a24c28e6fdd03ebfca26c23174b9a
  Stored in directory: /root/.cache/pip/wheels/ef/cf/7c/93e4012a84c44ffa5cf2a6a7ea03b282a8a19e17755808a9fb
Successfully built tensorrt tensorrt_cu12


In [38]:

!pip install nvidia-pyindex
!pip install onnx onnx_graphsurgeon tensorrt

import torch
import torch.onnx
from transformers import AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# Dummy input for tracing (adjust shape based on your model's expected input)
dummy_input = tokenizer("This is a test sentence.", return_tensors="pt")
dummy_input = {k: v.to(device) for k, v in dummy_input.items()} #add attention mask to dummy input

# Ensure model is in eval mode
model.eval()

# Export model to ONNX
torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']), # Pass both input_ids and attention_mask
    "model.onnx",
    export_params=True,
    opset_version=14,   # Change opset_version to 14
    do_constant_folding=True,
    input_names=["input_ids", "attention_mask"], # update input names
    output_names=["output"],
    dynamic_axes={"input_ids": {0: "batch_size"},"attention_mask":{0:"batch_size"}, "output": {0: "batch_size"}} #update dynamic_axes
)

print("✅ ONNX model saved as model.onnx")

import tensorrt as trt
import os

# Create logger and builder
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(TRT_LOGGER)

# Check if builder supports FP16 mode
if builder.platform_has_fast_fp16:
    print("✅ FP16 is supported and will be used.")
    config = builder.create_builder_config()
    config.set_flag(trt.BuilderFlag.FP16)
else:
    print("❌ FP16 is not supported.")
    config = builder.create_builder_config()


network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, TRT_LOGGER)

# Read ONNX model
with open("model.onnx", "rb") as f:
    if not parser.parse(f.read()):
        print("❌ Error parsing ONNX model")
        for i in range(parser.num_errors):
            print(parser.get_error(i))
        exit()

# Build optimized TensorRT engine
# config = builder.create_builder_config()

# Set workspace size using set_memory_pool_limit instead of max_workspace_size
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # Set workspace size (adjust if needed)

# Serialize the network
serialized_network = builder.build_serialized_network(network, config)

# Check if serialization was successful
if serialized_network is None:
    print("❌ Network serialization failed.")
else:
    print("✅ Network serialization successful.")
    # Save the TensorRT engine
    with open("model.trt", "wb") as f:
      f.write(serialized_network)
    print("✅ TensorRT engine saved as model.trt")

✅ ONNX model saved as model.onnx
✅ FP16 is supported and will be used.
❌ Network serialization failed.


In [39]:
# Check if builder supports FP16 mode
if builder.platform_has_fast_fp16:
    print("✅ FP16 is supported and will be used.")
    config = builder.create_builder_config()
    config.set_flag(trt.BuilderFlag.FP16)
else:
    print("❌ FP16 is not supported.")
    config = builder.create_builder_config()

✅ FP16 is supported and will be used.


In [40]:
# Build optimized TensorRT engine
# config = builder.create_builder_config()

# Set workspace size using set_memory_pool_limit instead of max_workspace_size
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # Set workspace size (adjust if needed)

# Serialize the network
serialized_network = builder.build_serialized_network(network, config)

# Check if serialization was successful
if serialized_network is None:
    print("❌ Network serialization failed.")
else:
    print("✅ Network serialization successful.")
    # Save the TensorRT engine
    with open("model.trt", "wb") as f:
      f.write(serialized_network)
    print("✅ TensorRT engine saved as model.trt")

❌ Network serialization failed.


In [42]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660393 sha256=409bdc9c8b90932451f696c3c91647d959a71106c057fe368c438b00a7e26cb9
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda
